# Latent Net implementation

In [1]:
# Install PyTorch
try:
  import torch
except ImportError:
  !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  import torch

torch.set_default_dtype(torch.float32)

In [2]:
# Install PyG
# try:
#   import torch_geometric
# except ImportError:
#   !pip3 install torch_geometric
#   import torch_geometric

In [3]:
import numpy as np

In [4]:
# Import some utils from gca-rom
import sys
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/Fra-Sala/gnn_time.git
    sys.path.append('gnn_time')
else:
    sys.path.append('./..')
    
from gca_rom import pde, scaling
import dynamics_network, initialization, loader, preprocessing_scale, train

In [5]:
variable = 'U'
problem_name = 'lid_cavity'
print("\nProblem: ", problem_name)
print("Variable: ", variable)



Problem:  lid_cavity
Variable:  U


In [6]:
# Parameters to be set

#preset = [3, 2, 2, 2, 1, 3, 3, 1]
train_rate = 70
dim_latent = 10
epochs = 5000
dt = 1e-2 # For forward euler
scaling_type = 2 # scale only against the features
scaler_number = 3 # type of scaling function. 3 -> standard

argv = [problem_name, variable, scaling_type, scaler_number, train_rate, 1e-3, 1e-3, dim_latent, dt, epochs]
HyperParams = dynamics_network.HyperParams(argv)

# Initialization

In [7]:
device = initialization.set_device()
initialization.set_reproducibility(HyperParams)

Device used:  cpu


# Load dataset

In [8]:
if 'google.colab' in str(get_ipython()):
    dataset_dir = '/content/gnn_time/dataset/'+problem_name+'_unstructured.mat'
else:
    dataset_dir = '../dataset/'+problem_name+'_unstructured.mat'

# Import the data from the mat file 
dataset = loader.LoadDataset(dataset_dir, variable)

# Delete the initial condition of each case
dataset.U = np.delete(dataset.U, np.s_[::10], 1)

In [9]:
# New approach for the parameters. Just store u(t) for all snapshots and the time t. if necessary, interpolate u(t) linearly

if 'google.colab' in str(get_ipython()):
    u_t_latent = np.load('/content/gnn_time/lid_driven_cavity_fenics/u_t_latent.npy')
else:
    u_t_latent = np.load('../lid_driven_cavity_fenics/u_t_latent.npy')

time = np.linspace(0.2, 1.8, 9)

# Get rid of the initial condition evaluation (0.0)
u_t_latent = np.delete(u_t_latent, np.s_[::10], 0)   
u_t_tensor = torch.from_numpy(u_t_latent)
time_tensor = torch.from_numpy(time)
print("Number of simulations:", len(u_t_latent))
print("Number of instants of time:", len(time))



Number of simulations: 180
Number of instants of time: 9


In [10]:
train_loader, test_loader, scaler_all,\
scaler_test, VAR_all, VAR_test, train_snapshots,\
test_snapshots, position_loader= preprocessing_scale.process_and_scale_dataset(dataset, HyperParams)

Number of nodes processed:  15681
Number of shapshots processed:  180


In [11]:
######### this is how you iterate over the positions
# for batch in position_loader:
#     x_batch, y_batch = batch
#     print(x_batch)


# Define the architecture

In [12]:
# The dynamics net takes u(t) as input and s(t)
dyn_input_size = 1 + HyperParams.dim_latent
dyn_hidden_size = 9
dim = 2 # we are in 2D

rec_input_size = dim*HyperParams.batch_size_pos + HyperParams.dim_latent
rec_hidden_size = dyn_hidden_size
rec_output_size = HyperParams.batch_size_pos


dyn_model = dynamics_network.DynNet(dyn_input_size, dyn_hidden_size, HyperParams.dim_latent)
rec_model = dynamics_network.RecNet(rec_input_size, rec_hidden_size, rec_output_size)
dyn_model = dyn_model.to(device)
rec_model = rec_model.to(device)

# Define optimizers for both models
dyn_optimizer = torch.optim.Adam(dyn_model.parameters(), lr=HyperParams.learning_rate,  weight_decay=HyperParams.weight_decay)
rec_optimizer = torch.optim.Adam(rec_model.parameters(), lr=HyperParams.learning_rate,  weight_decay=HyperParams.weight_decay)
dyn_scheduler = torch.optim.lr_scheduler.MultiStepLR(dyn_optimizer, milestones=HyperParams.miles, gamma=HyperParams.gamma)
rec_scheduler = torch.optim.lr_scheduler.MultiStepLR(rec_optimizer, milestones=HyperParams.miles, gamma=HyperParams.gamma)

# Train the network

In [13]:
# Need to properly define the training function

train.train_dyn_rec_nets(dyn_model, rec_model, dyn_optimizer, rec_optimizer, dyn_scheduler, rec_scheduler,\
                          device, u_t_tensor ,time_tensor, train_loader, test_loader, position_loader, train_snapshots, test_snapshots, HyperParams)

  0%|          | 0/5000 [00:00<?, ?it/s]


TypeError: cat() received an invalid combination of arguments - got (tuple, dtype=torch.dtype, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [ ]:
# Define your loss function (e.g., Mean Squared Error)
loss_fn = torch.nn.MSELoss()

# Training loop
for epoch in range(epochs):
    dyn_model.train()
    rec_model.train()
    total_loss = 0.0
    
    for i in range(len(params)):
        # Load data for the current parameter set
        u_t = parameters[i]  # 
        s_t = torch.zeros(HyperParams.dim_latent)  # Initialize s(t) with zeros

        for t in range(u_t.size(0)):
            # Forward pass through DynNet
            dyn_input = torch.cat((u_t[t], s_t), dim=0)
            s_t_derivative = dyn_model(dyn_input)

            # Compute s(tn+1) using forward Euler method
            s_t_plus_one = s_t + dt * s_t_derivative

            # Forward pass through RecNet
            rec_input = torch.cat((s_t_plus_one, position_x, position_y), dim=0)
            y_pred = rec_model(rec_input)

            # Calculate the loss
            target_velocity = dataset.U[i][t]  # Assuming V is the target velocity
            loss = loss_fn(y_pred, target_velocity)
            total_loss += loss.item()

            # Backpropagation and parameter updates for both models
            dyn_optimizer.zero_grad()
            rec_optimizer.zero_grad()
            loss.backward()
            dyn_optimizer.step()
            rec_optimizer.step()

            # Update s(t) for the next time step
            s_t = s_t_plus_one

    # Print the average loss for this epoch
    average_loss = total_loss / len(params)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {average_loss:.4f}")

    # Adjust the learning rates
    dyn_scheduler.step()
    rec_scheduler.step()

torch.save(dyn_model.state_dict(), 'dyn_model.pth')
torch.save(rec_model.state_dict(), 'rec_model.pth')

